# Player form

Here we are interested in evaluating how well the players have been doing recently, i.e. we evaluated each player's current form. Essentially, we calculate a weighted average of a player's adjusted points from a given number of previous games. We weight points so that more recent games are emphasized.

In [1]:
latest_gameweek = 34

# how many gameweeks do we use to evaluate form
number_games = 10

In [2]:
# import basic libraries

import pandas as pd
import numpy as np
pd.set_option('max_columns',100)

In [3]:
# fetch relevant data

data = []

for i in range(0, number_games+1):
    # create string for filename to be read
    data_string = '../data/data_week' + str(latest_gameweek-i) + '.csv'    
    try:
        # read data
        new_data = pd.read_csv(data_string,index_col='id')
        print(data_string)
        data.append(new_data) 
    # throw an exception if the data file is not found
    except FileNotFoundError:
        print(data_string + ' not found.')
    except UnicodeDecodeError:
        new_data = pd.read_csv(data_string, index_col='id', engine='python')
        print(data_string)
        data.append(new_data) 

../data/data_week34.csv
../data/data_week33.csv
../data/data_week32.csv
../data/data_week31.csv
../data/data_week30.csv
../data/data_week29.csv
../data/data_week28.csv
../data/data_week27.csv
../data/data_week26.csv
../data/data_week25.csv
../data/data_week24.csv


In [4]:
# this is used in the following to calculate form as a weighted average of recent gameweek adjusted points
def weighted_points(games_played,points,total_weight):
    weight = (number_games-i) / number_games
    total_weight += weight
    points += weight * (data[i].loc[ix,'adjusted points'] - data[i+1].loc[ix,'adjusted points']) / games_played
    return points, total_weight

In [5]:
# calculate form

column_name = 'form ' + str(number_games)
for ix in data[0][data[0]['minutes']>0].index:
    points = 0
    total_weight = 0
    for i in range(0, len(data)-1):
        if (ix in data[i].index) & (ix in data[i+1].index):
            # sometimes players get to play two games in a gameweek
            # also, it is possible that data from a given gameweek is missing, in which case data[i] and data[i+1] 
            # cover two gameweeks
            if (data[i].loc[ix,'minutes'] - data[i+1].loc[ix,'minutes']) > 90:
                points, total_weight = weighted_points(2,points,total_weight)
            elif (data[i].loc[ix,'minutes'] - data[i+1].loc[ix,'minutes']) > 0:
                points, total_weight = weighted_points(1,points,total_weight)
    if total_weight > 0:
        data[0].loc[ix, column_name] = points / total_weight

In [6]:
# display a sorted list showing players with the best current form
data[0].sort_values(by=column_name, ascending=False)[['web_name',column_name]]

,web_name,form 10
id,,
618,Fernandes,7.690343
113,Willian,7.144007
215,De Bruyne,7.017348
191,Salah,6.676319
219,David Silva,6.580889
431,Pulisic,6.068176
166,Vardy,6.009492
389,Antonio,5.940823
214,Sterling,5.747270


In [7]:
# save form results to latest data file

filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df = pd.read_csv(filepath, index_col=0)
df[column_name] = data[0][column_name].copy()
df.to_csv(filepath)